In [15]:
import xarray as xr
import numpy as np

In [23]:
ds_original = xr.open_dataset('chaz-no-random-frozen/2017_001-original-frozen.nc')
#ds_vectorized = xr.open_dataset('chaz-no-random-frozen/2017_001-vectorized-frozen.nc')

## most recent
ds_vectorized = xr.open_dataset('chaz-no-random-frozen/2017_001.nc')

In [24]:
def compare_xarray_outputs(ds_original, ds_vectorized):
    """
    Compare xarray datasets from original and vectorized versions.
    """
    print("=== XARRAY OUTPUT COMPARISON ===\n")
    
    # Extract the Mwspd variable (assuming this is the intensity)
    # Shape: (ensembleNum, lifelength, stormID)
    mwspd_orig = ds_original['Mwspd'].values
    mwspd_vec = ds_vectorized['Mwspd'].values
    
    print(f"Shape: {mwspd_orig.shape}")
    print(f"Dimensions: (ensembleNum={mwspd_orig.shape[0]}, lifelength={mwspd_orig.shape[1]}, stormID={mwspd_orig.shape[2]})")
    
    # Get lat/lon for basin identification
    lats = ds_original['latitude'].values[0, :]  # First time step
    lons = ds_original['longitude'].values[0, :]  # First time step
    
    # Compare ensemble 0 (deterministic component)
    print("\n=== ENSEMBLE 0 (Deterministic) ===")
    mwspd_orig_ens0 = mwspd_orig[0, :, :]  # Shape: (lifelength, stormID)
    mwspd_vec_ens0 = mwspd_vec[0, :, :]
    
    # 1. Check initialization (time=0)
    print("\nTime 0 (initialization):")
    diff_init = ~np.isclose(mwspd_orig_ens0[0, :], mwspd_vec_ens0[0, :], equal_nan=True)
    n_diff_init = np.sum(diff_init)
    print(f"  Number of differences: {n_diff_init}")
    if n_diff_init > 0:
        diff_storms = np.where(diff_init)[0]
        print(f"  First 5 differing storms: {diff_storms[:5]}")
        for iS in diff_storms[:3]:
            print(f"    Storm {iS}: orig={mwspd_orig_ens0[0, iS]:.2f}, vec={mwspd_vec_ens0[0, iS]:.2f}")
            print(f"      lat={lats[iS]:.2f}, lon={lons[iS]:.2f}")
    
    # 2. Check first few time steps
    print("\nFirst computed time steps:")
    for t in [2, 4, 6, 8, 10]:
        if t < mwspd_orig_ens0.shape[0]:
            diff_t = ~np.isclose(mwspd_orig_ens0[t, :], mwspd_vec_ens0[t, :], equal_nan=True)
            n_diff = np.sum(diff_t)
            print(f"  Time {t}: {n_diff} differences")
            if n_diff > 0 and t == 2:  # Show details for first difference
                diff_storms = np.where(diff_t)[0]
                print(f"    First 3 differing storms: {diff_storms[:3]}")
                for iS in diff_storms[:3]:
                    print(f"      Storm {iS}: orig={mwspd_orig_ens0[t, iS]:.2f}, vec={mwspd_vec_ens0[t, iS]:.2f}, diff={mwspd_orig_ens0[t, iS]-mwspd_vec_ens0[t, iS]:.2f}")
    
    # 3. By hemisphere
    print("\nBy Hemisphere (across all time):")
    nh_mask = lats >= 0
    sh_mask = lats < 0
    
    # Count total differences across all time steps
    diff_all_time = ~np.isclose(mwspd_orig_ens0, mwspd_vec_ens0, equal_nan=True)
    nh_diff_total = np.sum(diff_all_time[:, nh_mask])
    sh_diff_total = np.sum(diff_all_time[:, sh_mask])
    print(f"  Total NH differences: {nh_diff_total}")
    print(f"  Total SH differences: {sh_diff_total}")
    
    # 4. By basin
    print("\nBy Basin (time=2):")
    if mwspd_orig_ens0.shape[0] > 2:
        # North Atlantic
        na_mask = (lats > 0) & (lons >= -100) & (lons <= 0)
        # North Pacific  
        np_mask = (lats > 0) & (((lons >= 100) & (lons <= 180)) | ((lons >= -180) & (lons <= -100)))
        # NW Pacific
        nwp_mask = (lats >= 0) & (lons >= 120) & (lons <= 180)
        # Southern
        s_mask = lats < 0
        
        diff_t2 = ~np.isclose(mwspd_orig_ens0[2, :], mwspd_vec_ens0[2, :], equal_nan=True)
        print(f"  N.Atlantic: {np.sum(diff_t2 & na_mask)} differences")
        print(f"  N.Pacific: {np.sum(diff_t2 & np_mask)} differences")
        print(f"  NW.Pacific: {np.sum(diff_t2 & nwp_mask)} differences")
        print(f"  Southern: {np.sum(diff_t2 & s_mask)} differences")
    
    # 5. Show detailed time series for first problem storm
    print("\n=== DETAILED EXAMPLE ===")
    # Find first storm with differences
    diff_any_time = np.any(~np.isclose(mwspd_orig_ens0, mwspd_vec_ens0, equal_nan=True), axis=0)
    problem_storms = np.where(diff_any_time)[0]
    
    if len(problem_storms) > 0:
        iS = problem_storms[0]
        print(f"Storm {iS} (first storm with any differences):")
        print(f"  Location: lat={lats[iS]:.2f}, lon={lons[iS]:.2f}")
        print(f"  Time series (first 20 steps):")
        print(f"    Time | Original | Vectorized | Diff")
        for t in range(min(20, mwspd_orig_ens0.shape[0])):
            orig_val = mwspd_orig_ens0[t, iS]
            vec_val = mwspd_vec_ens0[t, iS]
            if not np.isnan(orig_val) and not np.isnan(vec_val):
                diff_val = orig_val - vec_val
                marker = " ***" if abs(diff_val) > 0.01 else ""
                print(f"    {t:4d} | {orig_val:8.2f} | {vec_val:8.2f} | {diff_val:8.2f}{marker}")
            else:
                print(f"    {t:4d} | {orig_val:8} | {vec_val:8} | NaN")
    else:
        print("No differences found!")
    
    # 6. Statistical summary
    print("\n=== STATISTICAL SUMMARY ===")
    # Only compare non-NaN values
    valid_mask = ~np.isnan(mwspd_orig_ens0) & ~np.isnan(mwspd_vec_ens0)
    if np.any(valid_mask):
        differences = mwspd_orig_ens0[valid_mask] - mwspd_vec_ens0[valid_mask]
        print(f"  Mean difference: {np.mean(differences):.4f}")
        print(f"  Max absolute difference: {np.max(np.abs(differences)):.4f}")
        print(f"  Std of differences: {np.std(differences):.4f}")
        print(f"  Number of non-zero differences: {np.sum(np.abs(differences) > 1e-6)}")
    
    return problem_storms

In [25]:
compare_xarray_outputs(ds_original, ds_vectorized)

=== XARRAY OUTPUT COMPARISON ===

Shape: (40, 125, 123)
Dimensions: (ensembleNum=40, lifelength=125, stormID=123)

=== ENSEMBLE 0 (Deterministic) ===

Time 0 (initialization):
  Number of differences: 1
  First 5 differing storms: [12]
    Storm 12: orig=nan, vec=25.00
      lat=-12.39, lon=127.87

First computed time steps:
  Time 2: 2 differences
    First 3 differing storms: [12 68]
      Storm 12: orig=nan, vec=28.82, diff=nan
      Storm 68: orig=45.29, vec=37.33, diff=7.96
  Time 4: 7 differences
  Time 6: 10 differences
  Time 8: 12 differences
  Time 10: 12 differences

By Hemisphere (across all time):
  Total NH differences: 646
  Total SH differences: 404

By Basin (time=2):
  N.Atlantic: 0 differences
  N.Pacific: 1 differences
  NW.Pacific: 1 differences
  Southern: 1 differences

=== DETAILED EXAMPLE ===
Storm 0 (first storm with any differences):
  Location: lat=-8.22, lon=50.11
  Time series (first 20 steps):
    Time | Original | Vectorized | Diff
       0 |    25.00 | 

array([  0,   4,   7,  12,  14,  18,  20,  21,  24,  31,  32,  33,  34,
        35,  41,  42,  47,  48,  53,  56,  58,  60,  61,  63,  66,  67,
        68,  70,  71,  74,  76,  79,  81,  84,  88,  89,  91,  93,  94,
        97,  98,  99, 100, 105, 108, 111, 115, 116, 121])

In [ ]:
def get_E1(bt):
    """
    get errors and residual of errors from predictand bt
    return:
      errors(E1), 
      [v0E],
      ranges of v0 (cat1)
    """

    NS = np.where((bt['StormYear']>=1981)&(bt['StormYear']<=2012))[0]

    data  = bt['errors'][:, NS].values          # shape (T, N)
    data2 = bt['StormMwspd'][:, NS].values      # shape (T, N)

    data1 = data.ravel(order = 'F')
    data4 = data2.ravel(order = 'F')

    ## mask out NaNs in data1
    nan_mask = ~np.isnan(data1)

    E1  = data1[nan_mask]
    E0  = data1[nan_mask]      
    v0E = data4[nan_mask]

    E2 = E0 * E1 * v0E
    mask = (E2 == E2)
    E0 = E0[mask]
    E1 = E1[mask]
    v0E = v0E[mask]
    c11, c0 = np.polyfit(E0, E1, 1)

    vals = v0E
    vmin = vals.min()
    vmax = vals.max()

    ## threshold grids 
    grid_up = np.arange(vmin, vmax, 10)       # for c1 (min → max)
    grid_dn = np.arange(vmax, vmin, -10)      # for c2 (max → min)

    ### c1 
    ## for every threshold iC in grid_up, compute how many values satisfy v0E < iC
    ## (n_vals,1) < (1,n_thresholds)
    counts_lt = (vals[:, None] < grid_up[None, :]).sum(axis=0)

    ## Pick first threshold where count > 50
    idx = np.argmax(counts_lt > 50) if np.any(counts_lt > 50) else len(grid_up)-1
    c1 = grid_up[idx]

    ### c2 
    # for every threshold iC in grid_dn, compute how many values satisfy v0E > iC
    counts_gt = (vals[:, None] > grid_dn[None, :]).sum(axis=0)

    idx = np.argmax(counts_gt > 50) if np.any(counts_gt > 50) else len(grid_dn)-1
    c2 = grid_dn[idx]

    ## final range
    cat1 = np.arange(c1, c2 + 10, 10)

    return E0,v0E,cat1

def findError(errort, v0E, v0, cat1):
    '''
    Calculate error 
    New version: reduces if statement trees. 
    '''

    if np.isnan(v0):
        return 0
    
    ## find once using searchsorted (faster than find_range)
    if v0 <= cat1[0]:
        mask = v0E < cat1[1]
    elif v0 >= cat1[-1]:
        mask = v0E >= cat1[-1]
    else:
        # Use searchsorted instead of find_range - it's faster
        j0 = np.searchsorted(cat1, v0, side='right') - 1
        mask = (v0E >= cat1[j0]) & (v0E < cat1[j0+1])
    
    error1 = errort[mask]
    ## FREEZING ERROR
    #return rng.choice(error1)
    return error1[0]

from bisect import bisect_left, bisect_right

def find_range(array, a):
    """
    Return two indices in which a is in between in an array
    array has to be a monotonic increase or decrease array
    """
    if (np.diff(array)<0).any():
        start = bisect_right(array[::-1],a)
        end = bisect_left(array[::-1],a)
        end = array.shape[0]-start
        start = end
    else:
        start = bisect_right(array,a)
        end = bisect_left(array,a)
    return (start-1, end)

## trying to numba speedup getPrediction
## doesn't improve efficiency very much, but doesn't slow it down, still tinkering

@numba.jit(nopython=True, cache=True)
def _compute_shrd(UShear, VShear):
    """Compute shear magnitude"""
    return np.sqrt(UShear**2 + VShear**2)

@numba.jit(nopython=True, cache=True)
def _normalize_and_accumulate(rvar, meanX_val, stdX_val, result_val, dy):
    """Normalize variable and add to accumulator"""
    rvar_norm = (rvar - meanX_val) / stdX_val
    return dy + rvar_norm * result_val

def getPrediction_v0input_result(bt,meanX,meanY,stdX,stdY,it,
                                 index,fstperiod,result,predictors,
                                 TimeDepends,v0,dvdt):
    v1 = []
    h1 = []
    v1.append(v0)
    h1.append(bt.Time[it,index])
    
    for ih in fstperiod:
        h1.append(bt.Time[it,index]+timedelta(hours = ih))
        lT = int(ih/6)
        dy = result[0]
        count2 = 0
        
        ## reduces number of nanmean calls slightly 
        piwspd_mean = nanmean(bt.PIwspdMean[it:it+lT+1,index])

        for var in predictors:
            #print meanX.shape
            ## change to == instead of 'in' for robustness
            if var == 'SHRD':
                ## use numba optimized function for SHRD calculation
                ushear = bt.UShearMean[it:it+lT+1,index]
                vshear = bt.VShearMean[it:it+lT+1,index]
                rvar = nanmean(_compute_shrd(ushear, vshear))
                #rvar = _nanmean(_compute_shrd(ushear, vshear))
            elif var == 'dPIwspd':
                #rvar = nanmean(bt.PIwspdMean[it:it+lT+1,index])-v0
                #rvar = _nanmean(bt.PIwspdMean[it:it+lT+1,index])-v0
                rvar = piwspd_mean-v0
            elif var in TimeDepends:
                rvar = nanmean(getattr(bt,var)[it:it+lT+1,index])
                #rvar = _nanmean(getattr(bt,var)[it:it+lT+1,index])
            elif var == 'StormMwspd':
                rvar = v0
            elif var == 'dVdt':
                rvar = dvdt
            elif var == 'landmaskMean':
                rvar = getattr(bt,var)[it+lT+1,index]-getattr(bt,var)[it,index]
            elif var == 'StormMwspd2':
                rvar = (getattr(bt,'StormMwspd')[it,index])**2
            elif var == 'StormMwspd3':
                rvar = (getattr(bt,'StormMwspd')[it,index])**3
            elif var == 'MPI':
                rvar = piwspd_mean
            elif var == 'MPI2':
                rvar = piwspd_mean**2
            elif var == 'dPIwspd2':
                rvar = (piwspd_mean-v0)**2
            elif var == 'dPIwspd3':
                rvar = (piwspd_mean-v0)**3
            elif var == 'dVdt2':
                rvar = dvdt**2
            elif var == 'dVdt3':
                rvar = dvdt**3
            else:
                rvar = getattr(bt,var)[it,index]

            ## normalize and accumulate using numba
            dy = _normalize_and_accumulate(rvar, meanX[count2+1], 
                                           stdX[count2+1], 
                                           result[count2+1], dy)
            count2 = count2 + 1
            
        dy = (dy*stdY)+meanY
        v1.append(v0+dy)
    return h1,v1

def get_landmask_value(bt, it, lT, iS):
    '''
    created this to remove duplicate code in get_determin and get_stochastic
    '''

    # using a single lookup instead of two
    ## slightly faster without optimized dask
    ## not sure about all edge cases 
    landmask_it = bt.landmaskMean[it,iS]
    landmask_itlT = bt.landmaskMean[it+lT,iS]
    
    # further simplifying boolean logic
    if landmask_itlT <= -0.5 and landmask_it <= -0.5:
        return result_w, meanX_w, meanY_w, stdX_w, stdY_w, \
               ['StormMwspd','dVdt','trSpeed','dPIwspd','SHRD','rhMean','dPIwspd2','dPIwspd3','dVdt2']
    
    ## assuming no other combination available except
    ## landmask_itlT > -0.5 and landmask_it > -0.5
    else:
        return result_l, meanX_l, meanY_l, stdX_l, stdY_l, \
               ['StormMwspd','dVdt','trSpeed','dPIwspd','SHRD','rhMean','dPIwspd2','dPIwspd3','dVdt2','landmaskMean']


def get_determin(iiS, block_id=None):
    '''
    This function finds the deterministic component of the autoregressive TC intensity model.
    iiS: bt data for intensity model

    '''
    iS = np.int_(iiS.mean(keepdims=True))
    
    lT = 2
    
    ## pre-extract storm's data once
    ## doesn't save that much time, but a little more readable
    StormLon_iS = bt.StormLon[:,iS]
    StormLat_iS = bt.StormLat[:,iS]
    landmaskMean_iS = bt.landmaskMean[:,iS]
    
    dummy = StormLon_iS[StormLon_iS==StormLon_iS]
    TimeDepends = ['dThetaEsMean','T200Mean','rhMean','rh500_300Mean','div200Mean']
    
    if ((dummy.any()) and (np.abs(StormLat_iS[0])>=5.)):
        it1 = 0
        it2 = np.int_(np.min([np.argwhere(StormLon_iS==dummy[-1])[-1,0],bt.StormLon.shape[0]-5]))
        
        if ((StormLat_iS[0] >=0) and (bt.StormLon[0,iS]>=120) and (bt.StormLon[0,iS]<=180)):
            #bt.determin[0,iS] = np.max([20, rng.choice(intV[intV==intV])])

            ## FREEZING RANDOM
            bt.determin[0, iS] = np.max([20, intV[intV==intV][0]])

        else:
            #bt.determin[0,iS] = np.max([25, rng.choice(intV[intV==intV])])

            ## FREEZING RANDOM
            bt.determin[0, iS] = np.max([25, intV[intV==intV][1]])
        
        dvdt = 0.0
        ih = 12
        lT = np.int_(ih/6)
        v0 = bt.determin[0,iS]
        
        ## made the if statements slightly more legible
        for it in range(it1,it2+2,2):
            if (
                it + lT < bt.StormLon.shape[0]
                and not np.isnan(StormLon_iS[it])
                and not np.isnan(landmaskMean_iS[it])
                and not np.isnan(landmaskMean_iS[it + lT])
                and not np.isnan(v0)
            ):
                ## code was identical between get_determin and get_stochastic, 
                ## defined a new function 
                ## may also help with landmask adjustment later
                result, meanX, meanY, stdX, stdY, predictors = get_landmask_value(bt, it, lT, iS)
                
                h1,v1 = getPrediction_v0input_result\
                         (bt,meanX,meanY,stdX,stdY,it,\
                         iS,[ih],result,predictors,\
                         TimeDepends,v0,dvdt)
                
                v1 = v1[1]
                if v1 < 10:
                    v1 = np.float64('Nan')
                    break
                bt.determin[it+lT,iS] = v1
                if ((bt.determin[it1:it+lT:lT,iS].max()>35) and (bt.determin[it,iS]<=35) and (bt.determin[it-lT,iS]<=35)):
                    break
                dvdt = v1-v0
                v0 = v1

        ## vectorized
        iit_indices = np.arange(it1+1, min(it2+1, bt.StormLon.shape[0]-2), 2)
        prev_vals = bt.determin[iit_indices-1, iS]
        next_vals = bt.determin[iit_indices+1, iS]

        mask = ~np.isnan(prev_vals * next_vals)  # true where neither is NaN

        bt.determin[iit_indices[mask], iS] = 0.5 * (prev_vals[mask] + next_vals[mask])
    
    return iS

def get_stochastic(iiS,block_id=None):
    '''
    This function finds the stochastic component of the autoregressive TC intensity model.
    iiS: bt data for intensity model
    '''
    TimeDepends = ['dThetaEsMean','T200Mean','rhMean','rh500_300Mean','div200Mean']
    iS = np.int_(iiS.mean(keepdims=True))
    dummy = bt.StormLon[:,iS][bt.StormLon[:,iS]==bt.StormLon[:,iS]]
    if ((dummy.any()) and (np.abs(bt.StormLat[0,iS])>=5.)):
        it1 = 0
        it2 = np.min([np.argwhere(bt.StormLon[:,iS]==dummy[-1])[-1,0],bt.StormLon.shape[0]-5])
        
        if ((bt.StormLat[0,iS] >=0) and (bt.StormLon[0,iS]>=120) and (bt.StormLon[0,iS]<=180)):
            ## update to use set rng
            #bt.stochastic[0, iS, iNN] = np.max([20, rng.choice(intV[intV==intV])]) #kts
            
            ## FREEZING RANDOM
            bt.stochastic[0, iS, iNN] = np.max([20, intV[intV==intV][0]])

        else:
            #bt.stochastic[0, iS, iNN] = np.max([25, rng.choice(intV[intV==intV])]) #kts

            ## FREEZING RANDOM
            bt.stochastic[0, iS, iNN] = np.max([25, intV[intV==intV][1]])
            
        ih = 12
        lT = np.int_(ih/6)
        v0 = bt.stochastic[0, iS, iNN]
        dvdt = 0.
        for it in range(it1,it2+2,2):
            ## making if statement a bit more readable
            if (
                it + lT < bt.StormLon.shape[0]
                and not np.isnan(bt.StormLon[it, iS])
                and not np.isnan(bt.landmaskMean[it, iS])
                and not np.isnan(bt.landmaskMean[it + lT, iS])
                and not np.isnan(v0)
            ):
                
                result, meanX, meanY, stdX, stdY, predictors = get_landmask_value(bt, it, lT, iS)
   
                h1,v1 = getPrediction_v0input_result\
                     (bt,meanX,meanY,stdX,stdY,it,\
                     iS,[ih],result,predictors,\
                     TimeDepends,v0,dvdt)
                
                error = findError(E0,v0E,v0,cat1)
                bt.error[it,iS,iNN] = error
                v1 = v1[1]
                v1 = v1-error
                if v1 < 10:
                    break;
                
                #if ((it >= it1+4*lT) and (bt.stochastic[it,iS,iNN]<=35) and (bt.stochastic[it-lT,iS,iNN]<=35)):
                bt.stochastic[it+lT,iS,iNN] = v1
                
                if ((bt.stochastic[it1:it+lT:lT,iS,iNN].max()>35) and (bt.stochastic[it,iS,iNN]<=35) and (bt.stochastic[it-lT,iS,iNN]<=35)):
                    break;
                dvdt = v1-v0
                v0 = v1


        ## vectorized
        # iit indices (every 2 steps)
        iit_indices = np.arange(it1+1, min(it2+1, bt.StormLon.shape[0]-2), 2)
        # neighbor values
        prev_vals = bt.stochastic[iit_indices - 1, iS, iNN]
        next_vals = bt.stochastic[iit_indices + 1, iS, iNN]
        # mask where neither neighbor is NaN
        mask = ~np.isnan(prev_vals * next_vals)
        # assign averaged values where valid
        bt.stochastic[iit_indices[mask], iS, iNN] = 0.5 * (prev_vals[mask] + next_vals[mask])

    return iS

def load_nc_via_pooch(filename):
    """Load NetCDF file via pooch and return xarray Dataset."""
    path = pooch.retrieve(url=f"{path_data}/{filename}", known_hash=None)
    return xr.open_dataset(path)


def calIntensity(iy, ichaz):
    '''
    This function calculates the intesity using an autoregressive model (Lee et al. (2015, 2016a)).
    EXP: location of historical simulations defined in Namelist.py
    iy: year of current iteration in CHAZ.py
    ichaz: current ensemble iteration in CHAZ.py`
    '''

    ## load datasets 
    bt2 = load_nc_via_pooch(gv.obpath)
    observed_data_ds = load_nc_via_pooch("observed_data.nc")
    coefficient_meanstd_ds = load_nc_via_pooch("coefficient_meanstd.nc")
    result_w_ds = load_nc_via_pooch("result_w.nc")
    result_l_ds = load_nc_via_pooch("result_l.nc")

    global result_w, result_l, meanX_w, meanY_w, stdX_w
    global stdY_w, meanX_l, meanY_l, stdX_l, stdY_l
    global E0, v0E, cat1, intV

    ## convert all variables into numpy arrays 
    result_w = result_w_ds['params'].values
    result_l = result_l_ds['params'].values
    meanX_w = coefficient_meanstd_ds.meanX_w.values
    meanY_w = coefficient_meanstd_ds.meanY_w.values
    stdX_w  = coefficient_meanstd_ds.stdX_w.values
    stdY_w  = coefficient_meanstd_ds.stdY_w.values
    meanX_l = coefficient_meanstd_ds.meanX_l.values
    meanY_l = coefficient_meanstd_ds.meanY_l.values
    stdX_l  = coefficient_meanstd_ds.stdX_l.values
    stdY_l  = coefficient_meanstd_ds.stdY_l.values

    ## TODO change variable name to not overwrite NS below
    NS = np.where((bt2['StormYear']>=1981)&(bt2['StormYear']<=2012))[0]
    intV = bt2['StormMwspd'][:][0,NS].values
    E0, v0E,cat1, = get_E1(bt2)

    ## initialize bt
    nsto = gv.CHAZ_Int_ENS
    bt.__dict__['determin'] = np.zeros(bt.StormLon.shape)*np.float64('nan')
    bt.__dict__['stochastic'] = np.zeros([bt.StormLon.shape[0],bt.StormLon.shape[1],nsto])*np.float64('nan')
    bt.__dict__['error'] = np.zeros([bt.StormLon.shape[0],bt.StormLon.shape[1],nsto])*np.float64('nan')

    ## TODO change variable name to not overwrite NS above
    NS = np.arange(0,bt.StormYear.shape[0],1)
    nS = da.from_array(NS.astype(dtype=np.int32),chunks=(1,))

    new = da.map_blocks(get_determin,nS,chunks=(1,), dtype=nS.dtype)
    n = new.compute(scheduler='synchronous',num_workers=10)
    #n = new.compute(scheduler='single-threaded',num_workers=10)
    del new

    new =da.map_blocks(get_stochastic,nS,chunks=(1,), dtype=nS.dtype)
    global iNN
    for iNN in range(nsto):
        n = new.compute(scheduler='synchronous',num_workers=10)
        #n = new.compute(scheduler='single-threaded')
    del new

    gc.collect()

    return bt

## Keeping track of function descriptions

def getPrediction_v0input_result_vectorized(bt, meanX, meanY, stdX, stdY, it,
                                           storm_indices, fstperiod, result, predictors,
                                           TimeDepends, v0_array, dvdt_array):
    """
    Processes storms simultaneously
    storm_indices: array of storm indices to process (nStorms,)
    v0_array: initial velocities (nStorms,)
    dvdt_array: velocity changes (nStorms,)
    returns: v1_array of shape (nStorms,)
    """


def get_stochastic_vectorized(iNN):
    '''
    Fully vectorized version that processes all storms simultaneously at each timestep.
    This function finds the stochastic component of the autoregressive TC intensity model.
    iNN: ensemble member index
    Modifies bt in-place (uses global bt).
    '''

def findError_vectorized(errort, v0E, v0_array, cat1):
    '''
    Calculate error for multiple storms
    v0_array: shape (nStorms,)
    returns: shape (nStorms,)
    '''